In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go
%matplotlib inline

In [ ]:
data = pd.read_csv(r'../input/netflix-shows/netflix_titles.csv')
data.head()

## Adding a new column 'target_ages' based on the content 'rating'

In [ ]:
ratings_ages = {
    'TV-PG': 'Older Kids',
    'TV-MA': 'Adults',
    'TV-Y7-FV': 'Older Kids',
    'TV-Y7': 'Older Kids',
    'TV-14': 'Teens',
    'R': 'Adults',
    'TV-Y': 'Kids',
    'NR': 'Adults',
    'PG-13': 'Teens',
    'TV-G': 'Kids',
    'PG': 'Older Kids',
    'G': 'Kids',
    'UR': 'Adults',
    'NC-17': 'Adults'
}
data['target_ages'] = data['rating'].replace(ratings_ages)
data.head()

## Displaying a graph of the amount of content by country

In [ ]:
countries = data['country'].value_counts()[data['country'].value_counts(normalize=True)>0.005]
list_countries = list(countries.index)
plt.figure(figsize=(20,10))
plt.title('Amount of content by country', fontsize=20)
plt.tick_params(labelsize=15)
sns.barplot(y=countries.index, x=countries.values, alpha=0.6)
plt.show()

## No. of movies and TV shows

In [ ]:
content_type = data['type'].value_counts()
plt.figure(figsize=(10,6))
plt.bar(content_type.index, content_type.values, alpha=0.8)
plt.title('No of Movies and TV Shows', fontsize=15)
plt.show

## Ages based Rating Distribution

In [ ]:
def generate_rating_data(data):
    rating_data = data.groupby(['rating', 'target_ages']).agg({'show_id':'count'}).reset_index()
    rating_data = rating_data[rating_data['show_id']!=0]
    rating_data.columns = ['rating', 'target_ages', 'counts']
    rating_data = rating_data.sort_values('target_ages')
    return rating_data

In [ ]:
movie_rating_data = generate_rating_data(data[data['type']=='Movie'])
show_rating_data = generate_rating_data(data[data['type']=='TV Show'])
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'pie'},{'type':'pie'}]])
fig.add_trace(go.Pie(labels=movie_rating_data['target_ages'], values=movie_rating_data['counts']),
             row=1,col=1)
fig.add_trace(go.Pie(labels=show_rating_data['target_ages'], values=show_rating_data['counts']),
             row=1, col=2)
fig.update_traces(textposition='inside', hole=0.4, hoverinfo='label+percent+name')
fig.update_layout(
    title_text="Ages based rating distribution by type of context", 
    annotations = [dict(text='Movies', x=0.2, y=0.5, font_size=12, showarrow=False),
                  dict(text='TV Shows', x=0.82, y=0.5, font_size=12, showarrow=False)]
)
fig.show()